In [ ]:
# Python 3.6.0
# tensorflow 1.1.0
# Keras 2.0.4

import os
import os.path as path

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

MODEL_NAME = 'mnist_convnet'
EPOCHS = 1
BATCH_SIZE = 128


def load_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)
    return x_train, y_train, x_test, y_test



def build_model():
    
    
  
    model = Sequential ()


    model.add(Conv2D(32 , kernel_size=(5,5), strides = (1,1) ,activation = 'relu' , input_shape  = [28,28,1] ))
    model.add(MaxPooling2D(pool_size = (2,2) , strides=(2,2)))

    model . add(Conv2D(64 , (5,5) ,activation = 'relu' ))
    model.add(MaxPooling2D(pool_size=(2,2)))
  
    model.add(Flatten())
    model.add(Dense(1000 , activation='relu'))
    model.add(Dense(10 , activation = 'softmax'))
    return model


def train(model, x_train, y_train, x_test, y_test):
    model.compile(loss=keras.losses.categorical_crossentropy, \
                  optimizer=keras.optimizers.Adadelta(), \
                  metrics=['accuracy'])

    model.fit(x_train, y_train, \
              batch_size=BATCH_SIZE, \
              epochs=EPOCHS, \
              verbose=1, \
              validation_data=(x_test, y_test))


def export_model(saver, model, input_node_names, output_node_name):
    tf.train.write_graph(K.get_session().graph_def, 'out', \
        MODEL_NAME + '_graph.pbtxt')

    saver.save(K.get_session(), 'out/' + MODEL_NAME + '.chkp')

    freeze_graph.freeze_graph('out/' + MODEL_NAME + '_graph.pbtxt', None, \
        False, 'out/' + MODEL_NAME + '.chkp', output_node_name, \
        "save/restore_all", "save/Const:0", \
        'out/frozen_' + MODEL_NAME + '.pb', True, "")

    input_graph_def = tf.GraphDef()
    with tf.gfile.Open('out/frozen_' + MODEL_NAME + '.pb', "rb") as f:
        input_graph_def.ParseFromString(f.read())

    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

    with tf.gfile.FastGFile('out/opt_' + MODEL_NAME + '.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

    print("graph saved!")


def main():
    if not path.exists('out'):
        os.mkdir('out')

    x_train, y_train, x_test, y_test = load_data()

    model = build_model()

    train(model, x_train, y_train, x_test, y_test)

    export_model(tf.train.Saver(), model, ["conv2d_1_input"], "dense_2/Softmax")


if __name__ == '__main__':
    
    main()

In [ ]:
from keras import backend as K
from keras.models import load_model
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
import tensorflow as tf



weight_file_path = 'mnist.h5'
net_model = load_model(weight_file_path)
sess = K.get_session()
out_nodes = []
out_prefix="output_"

for i in range(len(net_model.outputs)):
    out_nodes.append(out_prefix + str(i + 1))
    tf.identity(net_model.output[i], out_prefix + str(i + 1))
constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), out_nodes)
graph_io.write_graph(constant_graph, '', 'output.pb', as_text=False)
print('saved the constant graph (ready for inference) at: ', osp.join('', 'output.pb'))